<a href="https://colab.research.google.com/github/VirtualGoat/Enhanced-CCTV-security-using-DL/blob/master/Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install imageAI

     |████████████████████████████████| 184kB 17.6MB/s 


In [2]:
from google.colab import drive
import os
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
def forFrame(frame_number, output_array, output_count, detected_copy):
    print("FOR FRAME " , frame_number)
#    print("Output for each object : ", output_array)
    print("Output count for unique objects : ", output_count)
    print("------------END OF A FRAME --------------")

def forSeconds(second_number, output_arrays, count_arrays, average_output_count, detected_copy):
    print("SECOND : ", second_number)
#    print("Array for the outputs of each frame ", output_arrays)
    print("Array for output count for unique objects in each frame : ", count_arrays)
    print("Output average count for unique objects in the last second: ", average_output_count)
    print("------------END OF A SECOND --------------")

def forMinute(minute_number, output_arrays, count_arrays, average_output_count, detected_copy):
    print("MINUTE : ", minute_number)
#    print("Array for the outputs of each frame ", output_arrays)
    print("Array for output count for unique objects in each frame : ", count_arrays)
    print("Output average count for unique objects in the last minute: ", average_output_count)
    print("------------END OF A MINUTE --------------")

In [5]:
yolo_path = '/content/drive/My Drive/Colab Notebooks/ImageAI/'
video_path = '/content/drive/My Drive/Colab Notebooks/ImageAI/'
execution_path = os.getcwd()
print(execution_path)


/content


In [7]:
use_model = "yolo.h5"
use_video="messi.mp4"
from imageai.Detection import VideoObjectDetection

analisys_video = "detected_" + use_video

detector = VideoObjectDetection()

# Iniciate model
detector.setModelTypeAsYOLOv3()      # To YOLOv3 model
#detector.setModelTypeAsRetinaNet()   # To Retinanet model
#detector.setModelTypeAsTinyYOLOv3()   # To TinyYOLOv3 model

detector.setModelPath( os.path.join(yolo_path , use_model))
detector.loadModel()

vid_path = detector.detectObjectsFromVideo(input_file_path=os.path.join( video_path, use_video),
                                           output_file_path=os.path.join(video_path, analisys_video),
                                           frames_per_second=30,
                                 #          per_second_function = forSeconds,
                                           per_frame_function = forFrame,
                                 #          per_minute_function= forMinute,
                                           minimum_percentage_probability=30,
                                           log_progress=True,
                                           return_detected_frame=True)
                                           
 
print(vid_path)

Using TensorFlow backend.














Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Processing Frame :  1
FOR FRAME  1
Output count for unique objects :  {'tv': 1, 'person': 16}
------------END OF A FRAME --------------
Processing Frame :  2
FOR FRAME  2
Output count for unique objects :  {'person': 15}
------------END OF A FRAME --------------
Processing Frame :  3
FOR FRAME  3
Output count for unique objects :  {'person': 17}
------------END OF A FRAME --------------
Processing Frame :  4
FOR FRAME  4
Output count for unique objects :  {'baseball glove': 1, 'person': 16}
------------END OF A FRAME --------------
Processing Frame :  5
FOR FRAME  5
Output count for unique objects :  {'person': 16}
------------END OF A FRAME --------------
Processing Frame :  6
FOR FRAME  6
Output count for unique objects :  {'person': 13}
------------END OF A FRAME --------------
Processing Frame :  7
FOR FRAME  7
Output count for unique objects :  {'tv': 1, 'person': 15}
--------